# 21. TF2 API 개요

## 21-1. TensorFlow2 API로 모델 구성하기
### TensorFlow2 API 알아보기
#### 1) TensorFlow2 Sequential Model

In [2]:
# import tensorflow as tf
# from tensorflow import keras

# model = keras.Sequential()
# model.add(__넣고싶은 레이어__)
# model.add(__넣고싶은 레이어__)
# model.add(__넣고싶은 레이어__)

# model.fit(x, y, epochs=10, batch_size=32)

`model = keras.Sequential()` <br/>
- Sequential 모델은 입력 1가지, 출력 1가지를 전제로 함.
- sequential하게 차곡차곡 add해서 쌓아 간다.

#### 2) TensorFlow2 Functional API

In [3]:
# import tensorflow as tf
# from tensorflow import keras

# inputs = keras.Input(shape=(__원하는 입력값 모양__))
# x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
# x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
# outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

# model = keras.Model(inputs=inputs, outputs=outputs)
# model.fit(x, y, epochs=10, batch_size=32)

`keras.Model`
- `Sequential Model`보다 일반적 
- `Sequential Model`은 이 `keras.Model`을 상속받은 사례일 뿐 <br/>

`Funtional`
: 입력과 출력을 규정함으로써 모델 전체를 규정한다는 뜻 (함수형으로 모델 구성하기) <br/>

`Input`과 Output을 규정하면 Model이 `inputs`와 `outputs`만으로 규정이 된다. <br/>

- Sequential Model과 다르게 `Functial API`를 통해 다중 입력/출력을 가지는 모델을 구성할 수 있다.

#### 3) TensorFlow2 Subclassing

In [4]:
# import tensorflow as tf
# from tensorflow import keras

# class CustomModel(keras.Model):
#     def __init__(self):
#         super(CustomModel, self).__init__()
#         self.__정의하고자 하는 레이어__()
#         self.__정의하고자 하는 레이어__()
#         self.__정의하고자 하는 레이어__()
        
#     def call(self, ):
#         x = self.__정의하고자 하는 레이어(x)
#         x = self.__정의하고자 하는 레이어(x)
#         x = self.__정의하고자 하는 레이어(x)
        
#         return x

# model = CustomModel()
# model.fit(x, y, epochs=10, batch_size=32)

`Subclassing`
- 제일 자유로운 모델링 진행할 수 있음
- `keras.Model`을 상속받은 모델 클래스를 만드는 것 (`Functional`과 다른 점 없음) <br/>

`keras.Model`
- `__init__()` 메서드 안에서 레이어 간 `forward propagation`을 구현한다.

## 21-2. TensorFlow2 API로 모델 작성하기: MNIST (1) Sequential API 활용
- 앞서 본 TensorFlow2의 다양한 High-level API를 활용해 이미지 문제를 풀어보자.

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
# 데이터 구성 부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

print(len(x_train), len(x_test))

60000 10000


#### ❓ newaxis
: numpy array의 차원을 늘려주기

In [7]:
# Sequential Model 구성하기

"""
Spec:
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [8]:
# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 35s 4ms/step - loss: 0.1064 - accuracy: 0.9674
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0329 - accuracy: 0.9895
Epoch 3/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0196 - accuracy: 0.9932
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0127 - accuracy: 0.9958
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0092 - accuracy: 0.9968
313/313 - 1s - loss: 0.0455 - accuracy: 0.9888


[0.045518577098846436, 0.9887999892234802]

## 21-3. TensorFlow2 API로 모델 작성하기: MNIST (2) Functional API 활용
- `keras.Model`을 직접 활용해야 하므로 `keras.Input`으로 정의된 input 및 output 레이어 구성을 통해 model을 만들자.

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [11]:
# Functioanl Model 구성하기

"""
Spec:
0. (28X28X1) 차원으로 정의된 Input
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [12]:
# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1054 - accuracy: 0.9680
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0357 - accuracy: 0.9887
Epoch 3/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0192 - accuracy: 0.9940
Epoch 4/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0137 - accuracy: 0.9955
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0096 - accuracy: 0.9969
313/313 - 1s - loss: 0.0423 - accuracy: 0.9889


[0.04229588806629181, 0.9889000058174133]

Sequential과 Functional은 큰 차이가 없네

## 21-4. TensorFlow2 API로 모델 작성하기: MNIST (3) Subclassing 활용
- `Subclassing`은 `keras.Model`을 상속받은 클래스를 만드는 것
- `__init__()` 메서드 안에서 레이어를 선언하고, `call()` 메서드 안에서 `forward propagation`을 구현하는 방식임
- `Functional` 방식과 비교하자면, `call()`의 입력이 Input이고, `call()`의 리턴값이 Output이다.

In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [14]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [15]:
# Subclassing을 활용한 Model을 구성해주세요.

"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
6. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

model = CustomModel()

In [16]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1062 - accuracy: 0.9680
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0339 - accuracy: 0.9897
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0198 - accuracy: 0.9936
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0126 - accuracy: 0.9960
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0085 - accuracy: 0.9974
313/313 - 1s - loss: 0.0458 - accuracy: 0.9880


[0.045776933431625366, 0.9879999756813049]

본직적으로 3가지 방법 모두 같다.

## 21-5. TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (1) Sequential API 활용
https://yeon22.tistory.com/174

In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
# 데이터 구성 부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [19]:
# Sequential Model을 구성해주세요.
"""
Spec:
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [20]:
# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 6s 3ms/step - loss: 3.6078 - accuracy: 0.1578
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9063 - accuracy: 0.2853
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5985 - accuracy: 0.3477
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3795 - accuracy: 0.3905
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2167 - accuracy: 0.4255
313/313 - 1s - loss: 2.6046 - accuracy: 0.3564


[2.604600191116333, 0.3564000129699707]

## 21-6. TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (2) Functional API 활용
- 마찬가지로 keras.Model을 직접 활용해야 하므로 keras.Input으로 정의된 input 및 output 레이어 구성을 통해 model을 구현해야 함

In [21]:
import tensorflow as tf
from tensorflow import keras

In [22]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [23]:
# Functional API를 활용한 Model을 구성해주세요.
"""
Spec:
0. (32X32X3) 차원으로 정의된 Input
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [24]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.5822 - accuracy: 0.1637
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.8641 - accuracy: 0.2923
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5719 - accuracy: 0.3520
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3637 - accuracy: 0.3952
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.1996 - accuracy: 0.4317
313/313 - 1s - loss: 2.5522 - accuracy: 0.3654


[2.5521655082702637, 0.3653999865055084]

## 21-8. TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용
- Subclassing은 keras.Model을 상속 받은 클래스를 만드는 것으로 init() 메서드 안에 레이어를 선언하고, call() 메서드 안에 forward propagation을 구현함을 기억하자!

In [25]:
import tensorflow as tf
from tensorflow import keras

In [26]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [28]:
# Subclassing을 활용한 Model을 구성해주세요.

"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
7. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2, 2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2, 2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
model = CustomModel()

In [29]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.6108 - accuracy: 0.1566
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9220 - accuracy: 0.2793
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6334 - accuracy: 0.3361
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4283 - accuracy: 0.3812
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2590 - accuracy: 0.4121
313/313 - 1s - loss: 2.5960 - accuracy: 0.3557


[2.5960018634796143, 0.35569998621940613]

## 21-8. GradientTape의 활용
### Automatic differentiation - GradientTape
위 3가지 방법들에서 동일했던 구성

In [ ]:
# # 모델 학습 설정
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(x_train, y_train, epochs=5)

#### 딥러닝의 모델 훈련 과정
1. Forward Propagation 수행 및 중간 레이어값 저장
2. Loss 값 계산
3. 중간 레이어값 및 Loss를 활용한 chain rule 방식의 backward propagation 수행
4. 학습 파라미터 업데이트 <br/>

- TF2 API에는 `model.fi()`이란 메서드 안에 모두 추상화되어 감추어져 있다.
- tensorflow에서 젝오하는 `tf.GradientTape`는 위와 같은 순전파(forward pass)로 진행된 모든 연산의 중간 레이어 값을 `tape`에 기록하고, 이를 이용해 gradient를 계산한 후 `tape`를 폐기함 <br/>

아래에선 이전 스텝에서 진행했던 학습을 `tf.GradientTape`을 이용해 변형해보자.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

여기까진 앞에서 다룬 Subclassing을 활용한 모델 작성법과 동일하다. <br/>
달라지는 부분은 `model.compile()`, `model.fit()`을 통해 손쉽게 진행햇던 학습 세팅 및 수행 부분

In [ ]:
# # 모델 학습 설정
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(x_train, y_train, epochs=5)

위와 같은 방식으로 loss, optimizer을 지정해주면 내부적으로 매 스텝 학습이 진행될 때마다 발생하는 loss 및 gradient가 어떻게 학습 파라미터를 업데이트하게 되는지를 지정해주는 작업이 `model.compile()` 안에서 자동으로 진행되었다. <br/> <br/>

아래 코드는 `tape.gradient()`를 통해 매스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후 `optimizer.apply_gradients()`를 통해 발생한 그래디언트가 업데이트 해야 할 파라미터 `model.trainable_variables`를 지정해주는 과정을 기술한 것


In [30]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

위와 같이 매 스텝 진행되는 학습의 실제 동작이 `train_step()` 메서드로 구현되었다.

In [ ]:
# model.fit(x_train, y_train, epochs=5, batch_size=32)

이 `model.fit()`으로 위와 같이 한 줄로 간단히 수행되던 실제 배치 학습 과정은, 매 스텝마다 위에서 구현했던 `train_step()`가 호출되는 과정으로 바꾸어 구현할 수 있다. <br/>
`model.fit()` 호출 시 결정되는 `batch_size`만 이번 스텝에서 결정해주면 됨

In [31]:
import time

def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            x_batch.append(x)
            y_batch.append(y)
            if step % batch_size == batch_size - 1:
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 1.9104
Epoch 1: last batch loss = 1.7795
Epoch 2: last batch loss = 1.6579
Epoch 3: last batch loss = 1.5141
Epoch 4: last batch loss = 1.4745
It took 85.83051419258118 seconds


이렇게 위에서 구현한 `train_model()` 메서드가 실은 우리가 그동안 사용했던 `model.fit()` 메서드와 기능적으로 동일함을 확인할 수 있다. <br/><br/>

이렇게 `tf.GradientTape()`를 활용하면 `model.compile()`과 `model.fit()` 안에 감추어져 있던 한 스텝의 학습 단계(위 예제에선 `train_step` 메서드)를 끄집어내 자유롭게 재구성할 수 있다. <br/> <br/>
그동안 흔히 다루었던 지도학습 방식과 다른 강화학습 or GAN(Generative Advasarial Network)의 학습을 위해선 `train_step` 메서드의 재구성이 필수적이므로 `tf.GradientTape()`의 활용법을 꼭 숙지해두자.

In [32]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp / len(y_test) # Accuracy

1/1 [==============================] - 2s 2s/step


0.3452

그래디언트를 활용할 필요가 없는 evaluation 단계는 기존 `model.predict()` 메서드를 다시 활용했다. 학습이 충분히 진행되지 않았으므로 최종 Accuracy는 신경쓰지 말자 !